In [10]:
import json
import matplotlib.pyplot as plt
import pandas as pd
import requests
import seaborn as sns
import os

POLYGON_API_KEY = os.getenv("polygon_api_key")
HEADERS = {
    'Authorization': 'Bearer ' + POLYGON_API_KEY 
}
BASE_URL = 'https://api.polygon.io/v2/aggs'


In [21]:
def get_fx_pairs_data(pg_tickers):
    start = '2023-01-01'
    end = '2023-01-31'
    multiplier = '1'
    timespan = 'minute'
    fx_url = f"range/{multiplier}/{timespan}/{start}/{end}?adjusted=true&sort=asc&limit=50000"
    fx_pairs_dict = {}
    for pair in pg_tickers:
        response = requests.get(
            f"{BASE_URL}ticker/{pair}/{fx_url}", 
            headers=HEADERS
        )
        print(response)
        #fx_pairs_dict[pair] = pd.DataFrame(response['results'])
    return fx_pairs_dict


def format_fx_pairs(fx_pairs_dict):
    for pair in fx_pairs_dict.keys():
        fx_pairs_dict[pair]['t'] = pd.to_datetime(fx_pairs_dict[pair]['t'],unit='ms')
        fx_pairs_dict[pair] = fx_pairs_dict[pair].set_index('t')
    return fx_pairs_dict


def create_str_index(fx_pairs_dict):
    fx_pairs_str_ind = {}
    for pair in fx_pairs_dict.keys():
        fx_pairs_str_ind[pair] = fx_pairs_dict[pair].set_index(
            fx_pairs_dict[pair].index.to_series().dt.strftime(
                '%Y-%m-%d-%H-%M-%S'
            )
        )
    return fx_pairs_str_ind


def create_returns_series(fx_pairs_str_ind):
    for pair in fx_pairs_str_ind.keys():
        fx_pairs_str_ind[pair]['rets'] = fx_pairs_str_ind[pair]['c'].pct_change()
    return fx_pairs_str_ind 


In [22]:
pg_tickers = ["C:EURUSD", "C:MXNZAR"]
fx_pairs_dict = get_fx_pairs_data(pg_tickers)
formatted_fx_dict = format_fx_pairs(fx_pairs_dict)
fx_pairs_str_ind = create_str_index(formatted_fx_dict)
fx_returns_dict = create_returns_series(fx_pairs_str_ind) 

<Response [404]>
<Response [404]>
